In [1]:
# Contractions take from http://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
CONTRACTIONS = { 
"ain't": "am not","aren't": "are not","can't": "cannot","can't've": "cannot have","'cause": "because",
"could've": "could have","couldn't": "could not","couldn't've": "could not have","didn't": "did not",
"doesn't": "does not","don't": "do not","hadn't": "had not","hadn't've": "had not have","hasn't": "has not",
"haven't": "have not","he'd": "he would","he'd've": "he would have","he'll": "he will","he's": "he is",
"how'd": "how did","how'll": "how will","how's": "how is","i'd": "i would","i'll": "i will","i'm": "i am",
"i've": "i have","isn't": "is not","it'd": "it would","it'll": "it will","it's": "it is","let's": "let us",
"ma'am": "madam","mayn't": "may not","might've": "might have","mightn't": "might not","must've": "must have",
"mustn't": "must not","needn't": "need not","oughtn't": "ought not","shan't": "shall not","sha'n't": "shall not",
"she'd": "she would","she'll": "she will","she's": "she is","should've": "should have","shouldn't": "should not",
"that'd": "that would","that's": "that is","there'd": "there had","there's": "there is","they'd": "they would",
"they'll": "they will","they're": "they are","they've": "they have","wasn't": "was not","we'd": "we would",
"we'll": "we will","we're": "we are","we've": "we have","weren't": "were not","what'll": "what will",
"what're": "what are","what's": "what is","what've": "what have","where'd": "where did","where's": "where is",
"who'll": "who will","who's": "who is","won't": "will not","wouldn't": "would not","you'd": "you would",
"you'll": "you will","you're": "you are"
}

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import pyspark
import numpy as np
import string
import re
from pyspark.sql.functions import isnan, when, count, col
import pyspark.sql.functions as F

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import CountVectorizer, IDF
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType, IntegerType, FloatType, DoubleType, LongType
from pyspark.sql import Row
import pickle
PUNCTUATION = set(string.punctuation)
STOPWORDS = set(stopwords.words('english'))

In [3]:
df = spark.read.json(
    's3://aws-logs-816063959671-us-east-1/data/tldr-training-data.jsonl')

NameError: name 'spark' is not defined

In [ ]:
df.cache()

In [ ]:
df.count()

In [ ]:
df.printSchema()

# Create a subset for testing

In [ ]:
subset = df.sample(withReplacement=False, fraction=0.1)
subset.cache()
subset.count()

# Investigating Columns

In [ ]:
subset.select('body','content', 'id', 'normalizedBody', 'subreddit', 'title').show(5)

In [ ]:
subset.select([count(when(isnan(c), c)).alias(c) for c in subset.columns]).show()

In [ ]:
subset.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in subset.columns]).show()

In [ ]:
print('{}% of the subset is missing reddit post titles'.format(
    round(subset.filter(subset["title"].isNull()).count()/subset.count()*100)))

## A few entries have null subreddits. Investigate further

#### Also, explore the difference between body, content, and normalized body

Going to look at each of these entries where subreddit is null and make sure nothing is wrong with them, as well as use this to understand the differences between textual features

In [ ]:
subreddit_null = subset.filter(subset["subreddit"].isNull()).toPandas()

In [ ]:
for i in range(5):
    print('index: ' + str(i))
    print('Body:')
    print(subreddit_null.loc[i,'body'])
    print('-'*60)
    print('Content:')
    print(subreddit_null.loc[i,'content'])
    print('-'*60)
    print('Normalized Body:')
    print(subreddit_null.loc[i,'normalizedBody'])
    print('-'*60)
    print('Summary:')
    print(subreddit_null.loc[i,'summary'])
    print('-'*60)
    print('-'*60)

These all happen to be spam posts, so for cleaning, all rows where subreddit is null should be dropped. 

In terms of what these textual features are:
* __body__ appears to be the original post, with the tl;dr (aka 'too long; didn't read', reddit's term for a summary)
* __content__ is a cleaned version of __body__, without the tl;dr
* __normalizedBody__ is a cleaned version of __body__ (with the tl;dr)
* __summary__ is simply the tl;dr

An approach for EDA should be to confirm that content length is the same length as the normalizedBody without the tl;dr/summary. Summary's with less than 2-3 words should be reviewed and potentially dropped, since I don't want to summarize entries with only a 2-3 words.

# Exploratory Data Analysis

### Exploring subreddits

In [ ]:
nunique_subreddit = df.select("subreddit").distinct().count()
total_len_subreddit = df.select("subreddit").count()
print('There are {} different subreddits, which is roughly {}% of the dataframe.'.format(
        nunique_subreddit, (round(nunique_subreddit/total_len_subreddit*100,2))))

In [ ]:
subset.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in subset.columns]).show()

There are a few null subreddits (which were identified as spam above), which should be dropped

In [ ]:
# Filters out all rows where subreddit is null
subset = subset.filter(subset.subreddit.isNotNull())

# Make all text lowercase

In [ ]:
for col in ['body', 'content', 'normalizedBody', 'subreddit', 'summary', 'title']:
    subset = subset.withColumn(col, F.lower(F.col(col)))
# Converts 'null' strings in the title column back to null values
subset = subset.withColumn('title', when(subset.title == 'null', F.lit(None)).otherwise(subset.title))

## Exploring summary length

In [ ]:
subset.describe('summary_len').show()

Some of these have a min of 1 word, which is far too short (even 2-5 words may be too short), while some have 377 words, which is too long.

# Filter out edits

Noticed that mony columns contain 'edits' of their original post, but it is in the summary. The best approach is likely to slice off any text after 'edit:', since such a large amount of posts contain this. Will create a new column to contain this edited information in case there is any useful information there

In [ ]:
print('{}% of the dataset contains edits'.format(
    round(subset[subset['summary'].rlike('(edit:|[^a-z]edit)')].count()/subset.count()*100,2)))

### Two approaches to take: 
 * remove all entries with 'edit'
 * split summaries before and after 'edit', and use imputation to evaluate with/without those rows
 
Going to attempt approach 2, which is more robust

In [ ]:
split_col = F.split(subset['summary'], '(edit:|[^a-z]edit)')
subset = subset.withColumn('edit', split_col.getItem(1))
subset = subset.withColumn('summary', split_col.getItem(0))

### Readjust summary_len column

The summaries are now shorter because 'edit' is gone. summary length now needs to be recalculated

In [ ]:
print(len(word_tokenize('go for it, nobody cares. \n ')))
word_tokenize('go for it, nobody cares. \n ')

It's going to be necessary to make a user defined function to properly get word length

In [ ]:
def word_length(string):
    tokens = word_tokenize(string)
    tokens = [w for w in tokens if w not in PUNCTUATION]
    return len(tokens)

In [ ]:
word_length('go for it, nobody cares. \n ')

In [ ]:
function = udf(word_length, LongType())
subset = subset.withColumn('summary_len_cleaned', function(subset.summary))

### Create Edit_len column

In [ ]:
# Fill null values with '' in edit column so udf does not fail
subset = subset.withColumn('edit', subset.edit).na.fill('')
# Creates edit_len column, number of words from 'edit'
function = udf(word_length, LongType())
subset = subset.withColumn('edit_len', function(subset.edit))
# Converts -1 in edit_len column to null
subset = subset.withColumn('edit_len', 
        when(subset.edit_len == -1, F.lit(0)).otherwise(subset.edit_len))
# Turn's the edit column into a bool/imputation
subset = subset.withColumn('edit', when(subset.edit.isNull(), F.lit(0)).otherwise(1))
# Count summary's containing 'edit:'
subset[subset['summary'].rlike('(edit:|[^a-z]edit )')].count()

In [ ]:
subset.select('summary_len','summary_len_cleaned','edit','edit_len').show()

### Examining summarys with less than 5 words

In [ ]:
print('{} summaries with 1 word'.format(subset.filter(subset.summary_len == 1).count()))
print('{} summaries with less than 5 words'.format(subset.filter(subset.summary_len < 5).count()))
print('{}% of all entries have less than 5 words'.format(
    round(subset.filter(subset.summary_len < 5).count()/subset.count()*100,2)))

In [ ]:
subset.select('summary').filter(subset.summary_len == 1).show(5)

In [ ]:
subset.select('summary').filter((subset.summary_len == 4)).show(5, truncate=False)

Length of 4 is getting better, but still too short

In [ ]:
subset.select('summary').filter((subset.summary_len == 5)).show(truncate=False)

Length of 5 words is a good minimum cut off for cleaning, especially since this is only 5% of our data

In [ ]:
# This code removes all rows where the summary contains less than 5 words
subset = subset.filter(subset.summary_len >= 5)

### Examining summarys with too many words

##### Show the largest word count summary

In [ ]:
subset.select('summary_len').orderBy(F.desc("summary_len")).first()

In [ ]:
subset.select('summary').orderBy(F.desc("summary_len_cleaned")).first()

This is clearly not the sort of summary we are hoping to create. 

In [ ]:
counts = []
for i in range(0, 100, 10):
    counts.append(subset.select('summary').filter((subset.summary_len_cleaned >= i)).count())

In [ ]:
x=[0,10,20,30,40,50,60, 70, 80, 90]

In [ ]:
plt.plot(x, counts)

In [ ]:
percent_n50 = round(subset.select('summary').filter((subset.summary_len >= 50)).count()/subset.count()*100,2)
percent_n100 = round(subset.select('summary').filter((subset.summary_len >= 100)).count()/subset.count()*100,2)

print('50-word cutoff: {}% of the data\n 100-word cutoff: {}% of the data'.format(percent_n50, percent_n100))

In [ ]:
subset.select('summary').filter((subset.summary_len == 100)).first()

In [ ]:
subset.select('summary').filter((subset.summary_len == 50)).first()

50 Words may be a good cutoff but still could be too many words. it is getting more difficult to quantify. Maybe a measure of the summary length containing x% of the body length would be a good additional measure.

In the meantime, in the model.py file, the cleaning function will include a variable to specify the length of summary

In [ ]:
subset = subset.filter((subset.summary_len_cleaned <= 50))

### Filter out summaries that have too close of a word length to the content

We are looking to create concise summaries from larger content. A 50% word cut-off may be a good start.

In [ ]:
subset.select('summary_len_cleaned', 'content_len').show()

In [ ]:
subset.select('content_len', 'summary_len_cleaned').filter(
    (subset.summary_len_cleaned <= subset.content_len*0.5)).count()

In [ ]:
subset.select('content_len', 'summary_len_cleaned').filter((
    subset.summary_len_cleaned <= subset.content_len*0.5)).show()

In [ ]:
# This removes all rows where the summary length is not less than 50% of the content length
subset = subset.filter(subset.summary_len_cleaned <= subset.content_len*0.5)

# Clean Text

We only want to clean the Reddit Post Content, not the summary, since we want the output of our model to be plain english.

In [ ]:
def clean_text(text, remove_stopwords=True):
    text = text.lower()
    
    if True:
        text = text.split()
        new_text = [CONTRACTIONS[w] if w in CONTRACTIONS else w for w in text]
        text = " ".join(new_text)
    
    tokens = word_tokenize(text)
    tokens = [w for w in tokens if w not in PUNCTUATION]
    if remove_stopwords==True:
        tokens = [w for w in tokens if w not in STOPWORDS]
    return tokens

In [ ]:
clean_text('testing it out', False)

In [ ]:
cleantext_udf = udf(clean_text, StringType())
subset = subset.withColumn('content_cleaned', cleantext_udf(subset.content, F.lit(True)))

In [ ]:
subset.select('content_cleaned').show(truncate=False)

# Summary

Great, the dataset is now cleaned and ready for analysis. src/util.py contains a function to automate exactly what was accomplished in this notebook, and is shown below

In [ ]:
def clean_text(text, remove_stopwords=True):
    text = text.lower()
    if True:
        text = text.split()
        new_text = [CONTRACTIONS[w] if w in CONTRACTIONS else w for w in text]
        text = " ".join(new_text)

    tokens = word_tokenize(text)
    tokens = [w for w in tokens if w not in PUNCTUATION]
    if remove_stopwords==True:
        tokens = [w for w in tokens if w not in STOPWORDS]
    return tokens

def word_length(string):
    tokens = word_tokenize(string)
    tokens = [w for w in tokens if w not in PUNCTUATION]
    return len(tokens)

def clean_data(df, n_words_summary=50, remove_stopwords=True):
    # Get rid of all rows where subreddit is null (these are spam)
    df = df.filter(df.subreddit.isNotNull())
    # Lowercase columns:
    for col in ['body','content','normalizedBody','subreddit','summary','title']:
        df = df.withColumn(col, F.lower(F.col(col)))
    # Converts 'null' strings in the title column back to null values
    df = df.withColumn('title', when(df.title == 'null', F.lit(None)).otherwise(df.title))

    # Creat edit(bool) and edit_len columns, while removing 'edit:%' from summary column
    split_col = F.split(df['summary'], '(edit:|[^a-z]edit)')
    df = df.withColumn('edit', split_col.getItem(1))
    df = df.withColumn('summary', split_col.getItem(0))
    function = udf(word_length, LongType())
    df = df.withColumn('summary_len', function(df.summary))
        # Creates edit_len column, number of words from 'edit'
    df = df.withColumn('edit', df.edit).na.fill('')
    df = df.withColumn('edit_len', function(df.edit))
        # Converts -1 in edit_len column to null
    df = df.withColumn('edit_len',
        when(df.edit_len == -1, F.lit(0)).otherwise(df.edit_len))
    df = df.withColumn('edit', when(df.edit.isNull(), F.lit(0)).otherwise(1))
    # Remove all rows where summary contains less than 5 words
    df = df.filter(df.summary_len >= 5)
    # Remove all rows where summary contains greater than n_words_summary words
    df = df.filter((df.summary_len <= n_words_summary))
    # Remove all rows where the summary length is not less than 50% of the content length
    df = df.filter(df.summary_len <= df.content_len*0.5)
    # Clean Content column
    cleantext_udf = udf(clean_text, StringType())
    df = df.withColumn('content', cleantext_udf(df.content, F.lit(remove_stopwords)))
    return df

In [ ]:
clean_data(subset).take(5)